# Day 21: Prompt Engineering Patterns - Part 3

In this notebook, we'll implement function calling, a powerful technique that allows language models to interact with external tools and APIs.

## Overview

We'll cover:
1. Setting up the environment
2. Defining a function with a JSON schema
3. Implementing a function calling loop
4. Testing the function calling implementation

## 1. Setting Up the Environment

First, let's install and import the necessary libraries.

In [ ]:
# Install required packages
!pip install openai python-dotenv requests

In [ ]:
import os
import openai
import json
from dotenv import load_dotenv

# Load environment variables (API keys)
load_dotenv()

# Set up OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if API key is available
API_KEY_AVAILABLE = openai.api_key and openai.api_key != "your-api-key-here"
print(f"OpenAI API Key Available: {API_KEY_AVAILABLE}")

## 2. Defining a Function and its Schema

We'll define a simple function to get the current weather and create a JSON schema to describe it to the model.

In [ ]:
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location."""
    # This is a mock function for demonstration purposes
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "22", "unit": "celsius"})

# Define the JSON schema for the function
weather_function_schema = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
        },
        "required": ["location"]
    }
}

## 3. Implementing the Function Calling Loop

Now, let's create a function that handles the interaction with the model, including executing the function call if the model requests it.

In [ ]:
def run_conversation(user_prompt):
    """Run a conversation with the model, handling function calls."""
    messages = [{"role": "user", "content": user_prompt}]
    tools = [weather_function_schema] # In newer OpenAI versions, this is `tools`
    
    if not API_KEY_AVAILABLE:
        print("API key not available. Simulating function call.")
        return f"Simulated response: The weather in London is 15 degrees Celsius."

    # First API call to see if a function needs to be called
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613", # Use a model that supports function calling
        messages=messages,
        functions=tools, # In newer versions, this is `tools`
        function_call="auto",  # Let the model decide
    )
    response_message = response["choices"][0]["message"]

    # Check if the model wants to call a function
    if response_message.get("function_call"):
        available_functions = {
            "get_current_weather": get_current_weather,
        }
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # Call the function
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit")
        )

        # Send the function response back to the model
        messages.append(response_message)  # Add the assistant's turn
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # Add the function's response
        
        # Second API call to get the final response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        return second_response["choices"][0]["message"]["content"]
    else:
        return response_message["content"]

## 4. Testing the Function Calling Implementation

Let's test our function calling implementation with a few different prompts.

In [ ]:
# Test prompts
test_prompts = [
    "What's the weather like in San Francisco?",
    "What's the weather in Tokyo in celsius?",
    "Tell me a fun fact about the sun." # This should not trigger a function call
]

# Run the tests
for prompt in test_prompts:
    print(f"User Prompt: {prompt}")
    final_response = run_conversation(prompt)
    print(f"Final Response: {final_response}")
    print("-" * 50)

## 5. Conclusion

In this notebook, we've implemented a basic function calling system. Here are the key takeaways:

1. **JSON Schemas**: We define our functions using JSON schemas to make them understandable to the model.
2. **Two-Step Process**: Function calling is typically a two-step process: the model first indicates its intent to call a function, and after the function is executed, the result is sent back to the model to generate a final response.
3. **Tool Integration**: This capability allows language models to interact with the real world, access external data, and perform actions, significantly expanding their usefulness.

This concludes our exploration of prompt engineering for Day 21. We've covered fundamental and advanced techniques, from zero-shot prompting to function calling, that are essential for building robust and effective language model applications.